In [1]:
import numpy as np
from numpy import load
import h5py
import pandas as pd
import torch
import lightning as L
from pytorch_lightning import LightningModule
from scipy import linalg
import scipy
from tqdm import tqdm
from itertools import product
import tensorflow as tf
from tensorflow import keras
from six.moves import cPickle
from sklearn.metrics import mean_squared_error
import gc
from Bio import motifs
import os
from pymemesuite import fimo
from pymemesuite.common import MotifFile, Sequence
from pymemesuite import fimo
from pymemesuite.fimo import FIMO
from Bio import SeqIO

In [2]:
import glob
import tempfile
import Bio

In [3]:
from PL_DeepSTARR import *
from seq_evals_improved import *
from helpers import *

In [4]:
samples_file_path = 'samples.npz'
deepSTARR_data = 'DeepSTARR_data.h5'
oracle_path = 'oracle_DeepSTARR_DeepSTARR_data.ckpt'

In [5]:
deepstarr = load_deepstarr(oracle_path)

/opt/anaconda3/lib/python3.12/site-packages/torch/nn/modules/lazy.py:181: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [6]:
#load data
x_test, x_synthetic, x_train = extract_data(samples_file_path, deepSTARR_data)

In [7]:
x_test_tensor = numpy_to_tensor(x_test)
x_synthetic_tensor = numpy_to_tensor(x_synthetic)
x_train_tensor = numpy_to_tensor(x_train)

<h2>Functional similarity: Conditional generation fidelity</h2>

In [15]:
#make predictions
y_hat_test, y_hat_syn = load_predictions(x_test_tensor, x_synthetic_tensor, deepstarr)

In [16]:
#calculate Conditional generation fidelity mse
activity1 = y_hat_syn
activity2 = y_hat_test
mse = conditional_generation_fidelity(activity1, activity2)
print(mse)

1.2618008


<h2>Functional similarity: Frechet distance</h2>

In [6]:
embeddings1 = get_penultimate_embeddings(deepstarr, x_test_tensor)

In [7]:
embeddings2 = get_penultimate_embeddings(deepstarr, x_synthetic_tensor)

In [11]:
mu1, sigma1 = calculate_activation_statistics(embeddings1)

In [12]:
mu2, sigma2 = calculate_activation_statistics(embeddings2)

In [13]:
distance = calculate_frechet_distance(mu1, sigma1, mu2, sigma2)

In [14]:
distance

0.32809665113524034

<h2>Functional similarity: Predictive distribution shift</h2>

In [6]:
predictive_distribution_shift(x_synthetic_tensor, x_test_tensor)

0.007824236293496212

<h2>Sequence similarity: Percent identity</h2>

In [11]:
x_synthetic_tensor2 = x_synthetic_tensor

In [13]:
percent_identity = calculate_cross_sequence_identity_batch(x_synthetic_tensor, x_synthetic_tensor2, batch_size=256)

100%|█████████████████████████████████████████| 161/161 [00:14<00:00, 11.25it/s]


In [14]:
max_percent_identity = np.max(percent_identity, axis=1)

In [15]:
global_max_percent_identity = np.max(max_percent_identity)

In [16]:
global_max_percent_identity

126

In [18]:
percent_identity = calculate_cross_sequence_identity_batch(x_synthetic_tensor, x_train_tensor, batch_size=4096)

100%|███████████████████████████████████████████| 11/11 [01:21<00:00,  7.42s/it]


In [19]:
max_percent_identity = np.max(percent_identity, axis=1)

In [20]:
global_max_percent_identity = np.max(max_percent_identity)

In [21]:
global_max_percent_identity

127

<h2>Sequence similarity: k-mer spectrum shift</h2>

In [6]:
X_test, X_syn = put_deepstarr_into_NLA(x_test_tensor, x_synthetic_tensor)

In [7]:
kmer_statistics(3, X_test, X_syn)

100%|██████████████████████████████████| 41186/41186 [00:04<00:00, 10256.48it/s]


(0.001412, 0.018828)

<h2>Sequence similarity: Discriminatability</h2>

In [7]:
data_dict = prep_data_for_classification(x_test_tensor, x_synthetic_tensor)

In [8]:
write_to_h5('Discriminatability.h5', data_dict)

<h4>Next steps:</h4>
<h5>1. Upload Discriminatability.h5 and train_deepstarr_ood_score.py to a server with a GPU</h5>
<h5>2. Run qsub Discriminatability_test.sh</h5>
<h5>3. Look at auroc values contained in the pickle files</h5>

In [19]:
values = pd.read_pickle('deepstarr_3_ood.pickle')

<h2>Compositional similarity: Motif enrichment</h2>

In [8]:
x_test, x_synthetic = put_deepstarr_into_NLA(x_test_tensor, x_synthetic_tensor)

In [9]:
x_synthetic_e = one_hot_to_seq(x_synthetic[:100])
x_test_e = one_hot_to_seq(x_test[:100])

100%|███████████████████████████████████████| 100/100 [00:00<00:00, 6681.27it/s]


In [10]:
create_fasta_file(x_synthetic_e,'sub_sythetic_seq.txt')
create_fasta_file(x_test_e,'sub_test_seq.txt')

In [11]:
motif_count_1 = motif_count('sub_test_seq.txt', 'JASPAR2024_CORE_non-redundant_pfms_meme.txt')
motif_count_2 = motif_count('sub_sythetic_seq.txt', 'JASPAR2024_CORE_non-redundant_pfms_meme.txt')

In [14]:
motif_count_1

{'MA0004.1': 0,
 'MA0069.1': 4,
 'MA0071.1': 4,
 'MA0074.1': 3,
 'MA0101.1': 5,
 'MA0107.1': 5,
 'MA0115.1': 4,
 'MA0116.1': 4,
 'MA0119.1': 2,
 'MA0130.1': 0,
 'MA0142.1': 7,
 'MA0149.1': 7,
 'MA0151.1': 0,
 'MA0155.1': 4,
 'MA0159.1': 6,
 'MA0163.1': 4,
 'MA0145.2': 3,
 'MA0468.1': 7,
 'MA0515.1': 7,
 'MA0258.2': 6,
 'MA0595.1': 10,
 'MA0596.1': 7,
 'MA0599.1': 13,
 'MA0604.1': 5,
 'MA0608.1': 1,
 'MA0613.1': 12,
 'MA0614.1': 13,
 'MA0636.1': 5,
 'MA0641.1': 5,
 'MA0042.2': 11,
 'MA0033.2': 11,
 'MA0153.2': 3,
 'MA0486.2': 6,
 'MA0653.1': 12,
 'MA0655.1': 11,
 'MA0660.1': 10,
 'MA0663.1': 6,
 'MA0665.1': 13,
 'MA0667.1': 14,
 'MA0669.1': 4,
 'MA0676.1': 5,
 'MA0678.1': 2,
 'MA0068.2': 7,
 'MA0512.2': 0,
 'MA0083.3': 3,
 'MA0009.2': 6,
 'MA0689.1': 4,
 'MA0691.1': 20,
 'MA0696.1': 2,
 'MA0713.1': 7,
 'MA0715.1': 16,
 'MA0724.1': 2,
 'MA0728.1': 3,
 'MA0729.1': 2,
 'MA0730.1': 5,
 'MA0731.1': 7,
 'MA0472.2': 8,
 'MA0736.1': 9,
 'MA0737.1': 5,
 'MA0741.1': 16,
 'MA0088.2': 3,
 'MA0758.1

In [12]:
pr = enrich_pr(motif_count_1,motif_count_2)

In [13]:
pr

PearsonRResult(statistic=0.8854303017505831, pvalue=0.0)

<h2>Compositional similarity: Motif co-occurrence</h2>

<h2>Compositional similarity: Attribution maps</h2>

<h2>Compositional similarity: Attribution consistency</h2>